In [1]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.utils import np_utils
from keras import backend as K
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import random
from keras.layers import LeakyReLU
#from keras.applications.vgg16 import VGG16
#from keras.applications.vgg16 import preprocess_input
from keras.models import Model
import sys
import tensorflow as tf
import pathlib
from shutil import copy
import tensorflow_io as tfio 

In [271]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [272]:
tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

### SKIP, FOR file class

In [3]:
#file name, below are classify images

#file_names = sorted(item.name for item in pathlib.Path('F:/xlbsData/data/a2/raw').glob('*/') if item.is_dir())

In [111]:

#stuff_path=[str(path) for path in list(pathlib.Path('F:/xlbsData/final/animal_100').glob('*/')) ]

In [107]:

#i=0
#k=1
#for stuff in stuff_path:
#    to_path='F:/xlbsData/data/a1/sin/'+file_names[i]
#    copy(stuff, to_path)
#    if(k%100==0):
#        i+=1
#    k=k+1


### create dictionarty

In [5]:
file_names = sorted(item.name for item in pathlib.Path('F:/xlbsData/data/a2/raw').glob('*/') if item.is_dir())
label_to_index = dict((name, index) for index, name in enumerate(file_names))

### create dataset

In [242]:
def preprocess1(path, label):
    # read image
    image_raw = tf.io.read_file(path['raw_input']) 
    image_sin = tf.io.read_file(path['sin_input'])
    #decode 
    #image_sin = tf.image.decode_jpeg(image_sin, channels=3)
    image_raw=tfio.experimental.image.decode_tiff(image_raw)
    image_sin = tfio.experimental.image.decode_tiff(image_sin)
    # rsize o the same input size of resnet 
    image_raw = tf.image.resize(image_raw, [100, 100]) 
    image_sin = tf.image.resize(image_sin, [100, 100])
    image_raw= image_raw[:,:,0:3]
    image_sin= image_sin[:,:,0:3]
     # normalize
    image_raw /= 255.0 
    image_sin /= 255.0 
    return {'raw_input':image_raw,'sin_input':image_sin}, label

In [243]:
def preprocess2(path, label):
    # read image
    image_raw = tf.io.read_file(path['raw_input']) 
    image_sin = tf.io.read_file(path['sin_input'])
    #decode 
    image_sin = tf.image.decode_jpeg(image_sin, channels=3)
    image_raw=tfio.experimental.image.decode_tiff(image_raw)
    # rsize o the same input size of resnet 
    image_raw = tf.image.resize(image_raw, [100, 100]) 
    image_sin = tf.image.resize(image_sin, [100, 100])
    image_raw= image_raw[:,:,0:3]
    image_sin= image_sin[:,:,0:3]
     # normalize
    image_raw /= 255.0 
    image_sin /= 255.0 
    return {'raw_input':image_raw,'sin_input':image_sin}, label

In [244]:
def get_dataset(raw,sin,target,number):
    #get the path to dataset
    raw_path_r = pathlib.Path(raw)
    #get the last two level of the whole path , like damage/111.jpeg or no_damage/11111.jpeg
    raw_path = list(raw_path_r.glob('*/*')) 
    #transform all path to string, preparing for reading 
    raw_path = [str(path) for path in raw_path]
        #get the path to dataset
    sin_path = pathlib.Path(sin)
    #get the last two level of the whole path , like damage/111.jpeg or no_damage/11111.jpeg
    sin_path = list(sin_path.glob('*/*')) 
    #transform all path to string, preparing for reading 
    sin_path = [str(path) for path in sin_path]
        #get the label, damage or no_damage
    label_names_train = sorted(item.name for item in raw_path_r.glob('*/') if item.is_dir())
    #transform damage to 0, no_damage to 1
    all_image_labels_train = [label_to_index[pathlib.Path(path).parent.name] for path in raw_path]
    #get the data path and label
    ds_train = tf.data.Dataset.from_tensor_slices(({'raw_input':raw_path,'sin_input':sin_path}, all_image_labels_train))
    #preprocess fuciton, transform the data path to data and other preprocess
    if (number==1):
        ds_train=ds_train.map(preprocess2)
    else:
        ds_train=ds_train.map(preprocess1)
    return ds_train

In [266]:
#ds_2=get_dataset(a2_raw,a2_sin,target)

In [265]:
#ds_2

In [264]:
#for element in ds_2.take(1).as_numpy_iterator():
##    print(list(element[0]['raw_input']))
 #   plt.imshow(element[0]['raw_input'])

In [245]:
a1_raw='E:/AAA/data/a1/raw'
a1_sin='E:/AAA/data/a1/sin'
a2_raw='E:/AAA/data/a2/raw'
a2_sin='E:/AAA/data/a2/sin'
a3_raw='E:/AAA/data/a3/raw'
a3_sin='E:/AAA/data/a3/sin'
a4_raw='E:/AAA/data/a4/raw'
a4_sin='E:/AAA/data/a4/sin'
a5_raw='E:/AAA/data/a5/raw'
a5_sin='E:/AAA/data/a5/sin'
target='E:/AAA/data/a1/raw'

In [251]:
ds_1=get_dataset(a1_raw,a1_sin,target,1)
ds_2=get_dataset(a2_raw,a2_sin,target,2)
ds_3=get_dataset(a3_raw,a3_sin,target,3)
ds_4=get_dataset(a4_raw,a4_sin,target,4)
ds_5=get_dataset(a5_raw,a5_sin,target,5)

In [270]:
ds12=ds_1.concatenate(ds_2)
ds123=ds12.concatenate(ds_3)
ds1234=ds123.concatenate(ds_4)
ds_all=ds1234.concatenate(ds_5)

In [273]:
ds_all_shuffle=ds_all.shuffle(10000,seed=777)

In [274]:
ds_train=ds_all_shuffle.skip(2000)
ds_t_v=ds_all_shuffle.take(2000)
ds_test=ds_t_v.take(1000)
ds_validation=ds_t_v.skip(1000)

In [263]:
#i=0
#for element in ds12345.as_numpy_iterator():
#    i+=1
#    print(i)
#print(i)

In [262]:
#for element in ds12345.take(1).as_numpy_iterator():
  #print(np.shape(element))
 #   print(element)

### model

In [268]:
base_model1_1=tf.keras.applications.ResNet50V2(pooling=max,include_top=False,input_shape=(100,100,3))
base_model1_2=tf.keras.applications.ResNet50V2(pooling=max,include_top=False,input_shape=(100,100,3))

In [288]:
base_model1_1._name = "res1"
base_model1_2._name="res2"

In [289]:
base_model1_2.summary()

Model: "res2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 106, 106, 3)  0           input_8[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 50, 50, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 52, 52, 64)   0           conv1_conv[0][0]                 
_______________________________________________________________________________________________

In [269]:
base_model2=tf.keras.applications.Xception(pooling=max,include_top=False,input_shape=(100,100,3))
base_model3=tf.keras.applications.VGG16(pooling=max,include_top=False,input_shape=(100,100,3))

In [275]:
for layer in base_model1_1.layers[:]:
  layer.trainable=False
for layer in base_model1_2.layers[:]:
  layer.trainable=False
for layer in base_model2.layers[:]:
  layer.trainable=False
for layer in base_model3.layers[:]:
  layer.trainable=False

In [276]:
for layer in base_model1_1.layers[-5:]:
    layer.trainable=True
for layer in base_model1_2.layers[-5:]:
    layer.trainable=True
for layer in base_model2.layers[-5:]:
    layer.trainable=True
for layer in base_model3.layers[-3:]:
    layer.trainable=True

In [277]:
#base_model2.summary()

In [293]:
inputs_raw = tf.keras.Input(shape=(100, 100,3),name='raw_input')
inputs_sin = tf.keras.Input(shape=(100, 100,3),name='sin_input')
x1 = base_model1_1(inputs_raw)
x2 = base_model1_2(inputs_sin)

concat = tf.keras.layers.concatenate([x1,x2])
    #x = base_model(inputs)
    #use the gloval average pooling

GlobalAveragePooling2D = tf.keras.layers.GlobalAveragePooling2D() 
x=GlobalAveragePooling2D(concat)
    #use dropout
x = tf.keras.layers.Dropout(0.5)(x)
    #add a layer with 128 nodes
x=tf.keras.layers.Dense(4096,activation='relu')(x)
    #use dropout
x = tf.keras.layers.Dropout(0.5)(x)
    #add a layer with 64 nodes
x=tf.keras.layers.Dense(4096,activation='relu')(x)

outputs=tf.keras.layers.Dense(20,activation='softmax')(x)
model = tf.keras.Model(inputs=[inputs_raw,inputs_sin], outputs=[outputs])#compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00007),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                metrics=['accuracy'])
    

#history = model.fit(ds_train_aug.shuffle(4096,reshuffle_each_iteration=False).batch(32).prefetch(2),
#    epochs=10,validation_data=ds_validation.batch(256).prefetch(2),callbacks=[callback])

In [294]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
raw_input (InputLayer)          [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
sin_input (InputLayer)          [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
res1 (Functional)               (None, 4, 4, 2048)   23564800    raw_input[0][0]                  
__________________________________________________________________________________________________
res2 (Functional)               (None, 4, 4, 2048)   23564800    sin_input[0][0]                  
____________________________________________________________________________________________

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,restore_best_weights=True)
history=model.fit(ds_train.batch(32).prefetch(buffer_size=AUTOTUNE), epochs = 150,verbose=1, validation_data=ds_validation.batch(32).prefetch(buffer_size=AUTOTUNE),callbacks=[callback])

Epoch 1/150
250/250 [==============================] - 39s 69ms/step - loss: 1.7371 - accuracy: 0.4710 - val_loss: 0.8407 - val_accuracy: 0.7470
Epoch 2/150
250/250 [==============================] - 23s 67ms/step - loss: 1.0434 - accuracy: 0.6656 - val_loss: 0.5644 - val_accuracy: 0.8370
Epoch 3/150
250/250 [==============================] - 23s 65ms/step - loss: 0.8210 - accuracy: 0.7410 - val_loss: 0.3550 - val_accuracy: 0.8870
Epoch 4/150
250/250 [==============================] - 23s 65ms/step - loss: 0.6532 - accuracy: 0.7872 - val_loss: 0.3010 - val_accuracy: 0.9000
Epoch 5/150
250/250 [==============================] - 24s 66ms/step - loss: 0.5500 - accuracy: 0.8210 - val_loss: 0.2078 - val_accuracy: 0.9390
Epoch 6/150
250/250 [==============================] - 24s 67ms/step - loss: 0.4664 - accuracy: 0.8438 - val_loss: 0.1647 - val_accuracy: 0.9500
Epoch 7/150
250/250 [==============================] - 34s 106ms/step - loss: 0.3904 - accuracy: 0.8723 - val_loss: 0.1243 - val_a